# 第10章: ベクトル空間法 (II)
第10章では，前章に引き続き単語ベクトルの学習に取り組む．

## 90. word2vecによる学習
81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [1]:
!head -n 5 ../../chapter09/work/enwiki-20150112-400-r100-10576-compound_replaced.txt

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

In [2]:
from gensim.models.word2vec import LineSentence, Word2Vec
from gensim.test.utils import datapath

sentences = LineSentence('../../chapter09/work/enwiki-20150112-400-r100-10576-compound_replaced.txt')
model = Word2Vec(sentences, size=100, window=5, min_count=10, sg=1, hs=0, negative=5, workers=4)
model.save('../work/word2vec_d100_w5_sg.model')

In [3]:
from gensim.models.word2vec import Word2Vec

def knock_90():
    model = Word2Vec.load("../work/word2vec_d100_w5_sg.model")
    print("United_States の単語ベクトル:")
    print(model.wv["United_States"])
    
    print("\nUnited_States と U.S の類似度:")
    print(model.wv.similarity("United_States", "U.S"))

    print("\nEngland と類似度の高い単語10件")
    print(model.wv.most_similar(positive=['England'], topn=10))
    
    print("\nSpain - Madrid + Athens と類似度の高い単語10件")
    print(model.wv.most_similar(positive=["Spain", "Athens"], negative=["Madrid"], topn=10))
    
knock_90()

United_States の単語ベクトル:
[-0.22973995  0.40200445  0.34421453  0.28919438  0.48147416  0.49443465
  0.15673855 -0.17244482 -0.0892456   0.04869807  0.05364675  0.68151325
  0.40667236 -0.27873564  0.49128044  0.10711592 -0.23499027  0.24820426
 -0.22678265 -0.00636051 -0.5007369   0.40460145  0.11214581 -0.02220128
  0.13866962 -0.6351761  -0.11856941  0.2152162   0.24434619 -0.14506237
  0.06293315 -0.32003585  0.6672128   0.6049493   0.13256231 -0.70506674
  0.02116568 -0.06013344  0.2243821   0.24869251  0.3659247  -0.05015497
  0.07481691 -0.03340689 -0.37099144 -0.57469934 -0.24273023  0.00329948
 -0.4537681   0.39686134  0.03681759 -0.4316198   0.47957566  0.31808162
  0.00414515  0.21066937 -0.16097122 -0.04350206  0.86520225 -0.15035571
 -0.204073   -0.40912622 -0.38133255 -0.20733488  0.2642907   0.7181345
  0.01918561 -0.00173116 -0.5413542   0.10316164  0.45388025 -0.18308556
  0.03505403 -0.04935111  0.1608874  -0.0776216  -0.03734412  0.28746337
 -0.27174684 -0.47784606 -0.0

## 91. アナロジーデータの準備
単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [4]:
def knock_91():
    with open("../data/questions-words.txt") as f_in, open("../work/questions-words-family.txt", "w") as f_out:
        is_family = False
        for line in f_in:
            if line.startswith(": family"):
                is_family = True
                continue
            if line.startswith(": ") and is_family:
                break
            if is_family:
                f_out.write(line)

knock_91()

## 92. アナロジーデータへの適用
91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [6]:
import pickle
from functools import lru_cache
from gensim.models.word2vec import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

with open('../../chapter09/work/word_vector.pkl', 'rb') as f:
    word_vectors_85 = pickle.load(f)
with open('../../chapter09/work/word_to_index.pkl', 'rb') as f:
    word_to_index_85 = pickle.load(f)
index_to_word_85 = { index:word for word,index in word_to_index_85.items() }

model_90 = Word2Vec.load("../work/word2vec_d100_w5_sg.model")


def get_similar_word_and_vector_knock_85(base_word, minus_word, plus_word):
    base_word_vector = word_vectors_85[word_to_index_85[base_word.lower()]]
    minus_word_vector = word_vectors_85[word_to_index_85[minus_word.lower()]]
    plus_word_vector = word_vectors_85[word_to_index_85[plus_word.lower()]]
    target_vector = base_word_vector - minus_word_vector + plus_word_vector
    
    cosine_similarities = cosine_similarity(word_vectors_85, target_vector.reshape(1, -1))
    most_similar_index = cosine_similarities.argmax()
    word = index_to_word_85[most_similar_index]
    score = cosine_similarities[most_similar_index][0]
    return word, score

def get_similar_word_and_vector_knock_90(base_word, minus_word, plus_word):
    return model_90.wv.most_similar(positive=[base_word, plus_word], negative=[minus_word], topn=1)[0]

def knock_92():
    with open("../work/questions-words-family.txt") as f_in, open("../work/questions-words-family-85.txt", "w") as f_out_85, open("../work/questions-words-family-90.txt", "w") as f_out_90:
        for line in f_in:
            minus, base, plus, answer = line.rstrip().split()
            try:
                pred_word_85, pred_similarity_85 = get_similar_word_and_vector_knock_85(base, minus, plus)
                f_out_85.write(" ".join([minus, base, plus, answer, pred_word_85, str(pred_similarity_85)]).rstrip() + "\n")
            except KeyError:
                pass
            
            try:
                pred_word_90, pred_similarity_90 = get_similar_word_and_vector_knock_90(base, minus, plus)
                f_out_90.write(" ".join([minus, base, plus, answer, pred_word_90, str(pred_similarity_90)]).rstrip() + "\n")
            except KeyError:
                pass
            
knock_92()

In [7]:
!head -n 5 ../work/questions-words-family-85.txt

boy girl brother sister brother 0.9368151100011339
boy girl brothers sisters brothers 0.8131522467553747
boy girl dad mom dad 0.7552971447268428
boy girl father mother father 0.9078753940438338
boy girl grandfather grandmother grandfather 0.7172331960659377


In [8]:
!head -n 5 ../work/questions-words-family-90.txt

boy girl brother sister nephew 0.729115903377533
boy girl brothers sisters sisters 0.7195799946784973
boy girl dad mom mum 0.8103965520858765
boy girl father mother mother 0.8068951368331909
boy girl grandfather grandmother granddaughter 0.7659761905670166


## 93. アナロジータスクの正解率の計算
92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

In [9]:
import pandas as pd

def calc_accuracy(file_path):
    df = pd.read_csv(file_path, names=("minus", "base", "plus", "answer", "pred", "similarity"), delim_whitespace=True)
    return sum(1 for pred, answer in zip(list(df["pred"]),list(df["answer"])) if pred == answer) / float(len(list(df["answer"])))
    
def knock_93():
    print("正解率 (Word2Vec): " + str(calc_accuracy("../work/questions-words-family-90.txt")))
    print("正解率 (PMI):" + str(calc_accuracy("../work/questions-words-family-85.txt")))
    
knock_93()

正解率 (Word2Vec): 0.5225
正解率 (PMI):0.045454545454545456


## 94. WordSimilarity-353での類似度計算
The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

## 95. WordSimilarity-353での評価
94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

## 96. 国名に関するベクトルの抽出
word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

## 97. k-meansクラスタリング
96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5として実行せよ．

## 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

## 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．